## Data Input

In [19]:
with open('../data/tiny_shakespeare.txt', 'r') as f:
    text = f.read()

In [20]:
print(len(text))

1115394


In [21]:
print(text[:400])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it 


## Character Level Tokenization

In [22]:
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [23]:
char_to_index = {ch: i for i, ch in enumerate(chars)}
index_to_char = {i: ch for i, ch in enumerate(chars)} 

encode = lambda s: [char_to_index[c] for c in s]
decode = lambda ids: ''.join([index_to_char[i] for i in ids])

input_txt = "hello world"
encoded_data = encode(input_txt)
decoded_data = decode(encoded_data)
print(encoded_data)
print(decoded_data)

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [24]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:400])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

## Train, Validation Split

In [25]:
train_split_percentage = 0.9
n = int(train_split_percentage * len(data))
train_data = data[:n]
validation_data = data[n:]

## Context Block

In [26]:
context_length = 8

print(train_data[:context_length + 1])
print(decode(train_data[:context_length + 1].numpy()))

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
First Cit


In [27]:
x = train_data[:context_length]
y = train_data[1:context_length + 1]

print(x)
print(y)
print()

for t in range(context_length):
    context = x[:t + 1] 
    target = y[t]
    print(f"input: {context}, target: {target}")
    print(f"input: {context}, target: {target}")

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])

input: tensor([18]), target: 47
input: tensor([18]), target: 47
input: tensor([18, 47]), target: 56
input: tensor([18, 47]), target: 56
input: tensor([18, 47, 56]), target: 57
input: tensor([18, 47, 56]), target: 57
input: tensor([18, 47, 56, 57]), target: 58
input: tensor([18, 47, 56, 57]), target: 58
input: tensor([18, 47, 56, 57, 58]), target: 1
input: tensor([18, 47, 56, 57, 58]), target: 1
input: tensor([18, 47, 56, 57, 58,  1]), target: 15
input: tensor([18, 47, 56, 57, 58,  1]), target: 15
input: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47
input: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47
input: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58
input: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58


## Batching

In [28]:
torch.manual_seed(42)
batch_size = 4
context_length = 8


def get_batch(split):
    data = train_data if split == "train" else validation_data
    ix = torch.randint(low=0, high=len(data) - context_length, size=(batch_size, ))
    x = torch.stack([data[i: i + context_length] for i in ix])
    y = torch.stack([data[i + 1: i + context_length + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")
print("inputs")
print(xb.shape)
print(xb)
print("targets")
print(yb.shape)
print(yb)

print('-' * 10)

for b in range(batch_size):
    for t in range(context_length):
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"input: {context}, target: {target}")

inputs
torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
targets
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
----------
input: tensor([57]), target: 1
input: tensor([57,  1]), target: 46
input: tensor([57,  1, 46]), target: 47
input: tensor([57,  1, 46, 47]), target: 57
input: tensor([57,  1, 46, 47, 57]), target: 1
input: tensor([57,  1, 46, 47, 57,  1]), target: 50
input: tensor([57,  1, 46, 47, 57,  1, 50]), target: 53
input: tensor([57,  1, 46, 47, 57,  1, 50, 53]), target: 60
input: tensor([1]), target: 58
input: tensor([ 1, 58]), target: 46
input: tensor([ 1, 58, 46]), target: 43
input: tensor([ 1, 58, 46, 43]), target: 56
input: tensor([ 1, 58, 46, 43, 56]), target: 43
input: tensor([ 1, 58, 46, 43, 56

## Model

In [29]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super(BigramLanguageModel, self).__init__()
        self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=vocab_size)

    def forward(self, idx, targets=None):
        """Logits in shape of (batch, time_dim, channel_dim) which is reshaped to 2d tensor for cross entropy loss. 
        `t` is the time dimension and `c` is channel dim each token.
        """
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            b, t, c = logits.shape
            logits = logits.view(b * t, c)
            targets = targets.view(b * t)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens: int):
        """Generates new token from previous token and sample top 1 from softmax probs for next id.
        """
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            ids_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, ids_next], dim=1)

        return idx

## Setup

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [31]:
model = BigramLanguageModel(vocab_size=vocab_size).to(device)
logits, loss = model(xb.to(device), yb.to(device))
print(logits.shape)
print(loss)


pred_token_idx = model.generate(torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=100)
print(pred_token_idx)
print(pred_token_idx.shape)
print(decode(pred_token_idx[0].tolist()))

torch.Size([32, 65])
tensor(4.8865, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[ 0, 22,  0,  9, 59,  1, 10, 37,  9, 30, 22, 58, 40, 64, 16, 28,  5, 34,
         18, 32, 61, 28, 19, 58,  5, 59, 37,  9, 33, 35, 42, 36, 30,  9, 24, 52,
         55, 32, 15, 33,  6, 36, 54, 58, 62, 41, 19, 37,  6, 59, 27, 46, 48, 38,
         28, 21, 51, 36, 17, 28,  7, 25, 13, 31,  7, 62, 23, 54, 19, 54, 27, 61,
         47, 57, 15, 40, 25, 21, 11, 21, 32,  9, 57, 16, 55, 56, 45, 38, 28, 54,
         58, 40, 13, 57, 58, 28, 22,  8, 35, 19, 21]], device='cuda:0')
torch.Size([1, 101])

J
3u :Y3RJtbzDP'VFTwPGt'uY3UWdXR3LnqTCU,XptxcGY,uOhjZPImXEP-MAS-xKpGpOwisCbMI;IT3sDqrgZPptbAstPJ.WGI


In [32]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [33]:
from contextlib import nullcontext
ctx = nullcontext() if device == 'cpu' else torch.amp.autocast(device_type=device, dtype=torch.float16)

## Multiple Batch Loss Evaluation 

In [34]:
batch_size = 32
train_iters = 10000
eval_iters = 1000

In [35]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            x = x.to(device)
            y = y.to(device)
            with ctx:
                logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()            
    return out

## Training

In [36]:
for i in range(train_iters):
    xb, yb = get_batch('train')
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if i % eval_iters == 0:
        # losses = estimate_loss()
        # print(f"step: {i}: train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

        x, y = get_batch('val')
        x = x.to(device)
        y = y.to(device)
        with ctx:
            val_logits, val_loss = model(x, y)

        print(f"step: {i}: train loss: {loss:.4f}, val loss: {val_loss:.4f}")


print(f"final train loss: {loss.item():.4f}, val loss: {val_loss:.4f}")


step: 0: train loss: 4.7235, val loss: 4.8366
step: 1000: train loss: 3.7480, val loss: 3.7540
step: 2000: train loss: 3.1858, val loss: 3.1581
step: 3000: train loss: 2.7646, val loss: 2.9920
step: 4000: train loss: 2.6448, val loss: 2.6794
step: 5000: train loss: 2.6049, val loss: 2.6654
step: 6000: train loss: 2.5604, val loss: 2.4489
step: 7000: train loss: 2.4623, val loss: 2.4588
step: 8000: train loss: 2.4680, val loss: 2.5116
step: 9000: train loss: 2.4063, val loss: 2.4581
final train loss: 2.5170, val loss: 2.4581


## Generation

In [37]:
pred_token_idx = model.generate(torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=400)
# print(pred_token_idx)
print(pred_token_idx.shape)
print(decode(pred_token_idx[0].tolist()))

torch.Size([1, 401])

cody inde Miced tlat mang yofowhas
SThind.
UCENGr url hathapr me machan fl haisu d iere--d'durore cense wince me, woass t avest t, metha'?

Mavy?


TINSThyourBRGLONGof ind s hevalshe, ENVI an ndowe:
LUSGo be's lore we, bd ph-erpefoue t he ososour talunghimybr g y Hockis thtin;
Bee s mire akns t
Thanesa amnfod bs as Y w:
K:
Tharease VQUK: my feristhatessotrilorule, maripakn 'sealanks, gZABETZWeide.
